In [ ]:
!pip install transformers==4.30.2
!pip install peft -U
!pip install torch


In [ ]:
!pip install accelerate -U
!pip install bitsandbytes -U

In [8]:
!unzip saved_model_0.zip
!unzip saved_model_1.zip
!unzip saved_model_2.zip

Archive:  saved_model_0.zip
   creating: content/finetuned_model_0/
  inflating: content/finetuned_model_0/adapter_model.safetensors  
   creating: content/finetuned_model_0/runs/
   creating: content/finetuned_model_0/runs/Mar21_21-31-09_206f311006a2/
  inflating: content/finetuned_model_0/runs/Mar21_21-31-09_206f311006a2/events.out.tfevents.1711056823.206f311006a2.10532.1  
  inflating: content/finetuned_model_0/README.md  
  inflating: content/finetuned_model_0/adapter_config.json  
Archive:  saved_model_1.zip
   creating: content/finetuned_model_1/
  inflating: content/finetuned_model_1/adapter_model.safetensors  
   creating: content/finetuned_model_1/runs/
   creating: content/finetuned_model_1/runs/Mar22_01-57-45_6c2d907036cd/
  inflating: content/finetuned_model_1/runs/Mar22_01-57-45_6c2d907036cd/events.out.tfevents.1711072848.6c2d907036cd.1909.1  
  inflating: content/finetuned_model_1/README.md  
  inflating: content/finetuned_model_1/adapter_config.json  
Archive:  saved_mod

In [9]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftConfig, PeftModel
import torch
import accelerate
import bitsandbytes

In [11]:
# Paths to your models
model_paths = ["./models/finetuned_model_0", "./models/finetuned_model_1", "./models/finetuned_model_2"]

In [12]:
model_name = "THUDM/chatglm2-6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, load_in_8bit=True, device="cuda")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
model = PeftModel.from_pretrained(model, model_paths[0], adapter_name='Institute_0')
_ = model.load_adapter(model_paths[1], adapter_name='Institute_1')
_ = model.load_adapter(model_paths[2], adapter_name='Institute_2')

In [14]:
adapters = ["Institute_0", "Institute_1", "Institute_2"]
weights = [1.0, 1.0, 1.0]
adapter_name = "merge"
density = 0.2
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type="linear")

In [15]:
model.set_adapter("merge")

In [16]:
# Save the new model
output_model_path = "./aggregated_model"
model.save_pretrained(output_model_path)

In [18]:
model.add_weighted_adapter(adapters, weights, "merge_ties", combination_type="ties", density=density)
output_model_path = "./aggregated_model"
model.save_pretrained(output_model_path)

In [19]:
!zip -r ./aggregated_model.zip ./aggregated_model

updating: aggregated_model/ (stored 0%)
updating: aggregated_model/README.md (deflated 66%)
updating: aggregated_model/Institute_2/ (stored 0%)
updating: aggregated_model/Institute_2/adapter_config.json (deflated 50%)
updating: aggregated_model/Institute_2/adapter_model.safetensors (deflated 7%)
updating: aggregated_model/Institute_1/ (stored 0%)
updating: aggregated_model/Institute_1/adapter_config.json (deflated 50%)
updating: aggregated_model/Institute_1/adapter_model.safetensors (deflated 7%)
updating: aggregated_model/merge/ (stored 0%)
updating: aggregated_model/merge/adapter_config.json (deflated 51%)
updating: aggregated_model/merge/adapter_model.safetensors (deflated 35%)
updating: aggregated_model/Institute_0/ (stored 0%)
updating: aggregated_model/Institute_0/adapter_config.json (deflated 50%)
updating: aggregated_model/Institute_0/adapter_model.safetensors (deflated 7%)
  adding: aggregated_model/merge_ties/ (stored 0%)
  adding: aggregated_model/merge_ties/adapter_config.j

### Inference

In [20]:
#clone the FinNLP repository
!git clone https://github.com/AI4Finance-Foundation/FinNLP.git

import sys
sys.path.append('/content/FinNLP/')

Cloning into 'FinNLP'...
remote: Enumerating objects: 1412, done.
remote: Counting objects: 100% (471/471), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 1412 (delta 244), reused 433 (delta 224), pack-reused 941
Receiving objects: 100% (1412/1412), 4.97 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (617/617), done.


In [22]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.0 MB/s eta 0:00:00


In [23]:
# Load benchmark datasets from FinNLP
import datasets
from finnlp.benchmarks.fpb import test_fpb
from finnlp.benchmarks.fiqa import test_fiqa , add_instructions
from finnlp.benchmarks.tfns import test_tfns
from finnlp.benchmarks.nwgi import test_nwgi

In [24]:
model.set_adapter("merge")


In [28]:
batch_size = 40

before: 79.8 units


In [29]:
# TFNS Test Set, len 2388
# Available: 84.85 compute units
res = test_tfns(model, tokenizer, batch_size = batch_size)
# Available: 83.75 compute units
# Took about 1 compute unite to inference



Prompt example:
Instruction: What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
Input: $ALLY - Ally Financial pulls outlook https://t.co/G9Zdi1boy5
Answer: 


Total len: 2388. Batchsize: 40. Total steps: 60


100%|██████████| 60/60 [1:11:13<00:00, 71.23s/it]

Acc: 0.6557788944723618. F1 macro: 0.26403641881638845. F1 micro: 0.6557788944723618. F1 weighted (BloombergGPT): 0.5194485324955582. 


After: 63.46 Units
Accuracy: 0.655
F1 Macro: 0.264
F1 micro: 0.656
F1 weighted (BloombergGPT): 0.51944